In [1]:
#Importing the necessary libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
plt.style.use('fivethirtyeight')
import re


In [3]:
#create an Authentication 
authenticate=tweepy.OAuthHandler('eRYN8HRoTXHs60jhcQVEQl5Yp','i4n6TdkC7aVqPlQCA4HaqwFeDl4fGK5OFQ6eCKqAL0xvO5NqTR')

In [4]:
#set access tokens 
authenticate.set_access_token('838298485090889728-QMiYZsCVPtPz7KcOqqxmPEjJhKMaYPo',
                              'Br6YxCtORLA1hXgbCg5tSa4KRHsYvfHDCLPc7KYbQsZdO')

In [5]:
#creating an API object by passing the authentication object
api=tweepy.API(authenticate,wait_on_rate_limit=True)

In [6]:
#fetching the recent 100 posts from bill gates's timeline
posts=api.user_timeline(screen_name="BillGates",count=1000,lan="en",tweet_mode="extended")

In [7]:
len(posts)

200

In [8]:
#printing first 5 tweets
for tweet in posts[:5]:
    print(tweet.full_text)
    print()

RT @UN: 6 months ago, Africa confirmed its first case of #COVID19. Since then, there have been 1 million cases, affecting every country on…

If we’re going to address climate change, we need to find new ways to do things that don’t release greenhouse gases—including how we move around. https://t.co/ElpoZs2ZHG

Today we move one step closer toward a world without polio. This is an incredible milestone that’s worth celebrating. It is also an important reminder of what can be accomplished when we work together toward a common goal. https://t.co/Exxks9jF4q

RT @gatesfoundation: Thanks to the commitment of leaders from all 47 countries in the @WHOAFRO Region, history was made today: the entire A…

Mosquito City is home to the world’s largest captive colony of mosquitoes. @IfakaraHealth Institute is doing fascinating research there about mosquito behaviors and finding ways to outsmart them. https://t.co/p3r11gMqdn



In [9]:
#creating the dataframe from the tweets
data=pd.DataFrame([tweet.full_text for tweet in posts],columns=['Tweets'])
data.head()

,Tweets
0,"RT @UN: 6 months ago, Africa confirmed its fir..."
1,"If we’re going to address climate change, we n..."
2,Today we move one step closer toward a world w...
3,RT @gatesfoundation: Thanks to the commitment ...
4,Mosquito City is home to the world’s largest c...


In [14]:
#clean the data

#creating a function to remove some of the unwanted stuffs
def clean_text(text):
    text=re.sub('@[a-zA-Z0-9]+','',text)
    text=re.sub('#[a-zA-Z0-9]+','',text)
    text=re.sub('RT[\s]+','',text)
    text=re.sub('https?:\/\/\S+','',text)
    text=re.sub('[^a-zA-Z0-9 ]','',text)
    return text

In [15]:
#applying the cleaning function to our dataframe 
data.Tweets=data.Tweets.apply(clean_text)
data.head()

,Tweets
0,6 months ago Africa confirmed its first case ...
1,If were going to address climate change we nee...
2,Today we move one step closer toward a world w...
3,Thanks to the commitment of leaders from all ...
4,Mosquito City is home to the worlds largest ca...


In [16]:
data.head(10)

,Tweets
0,6 months ago Africa confirmed its first case ...
1,If were going to address climate change we nee...
2,Today we move one step closer toward a world w...
3,Thanks to the commitment of leaders from all ...
4,Mosquito City is home to the worlds largest ca...
5,Deaths from malaria have been cut by more than...
6,Vaccines prevent millions of deaths every yea...
7,Ridding the world of preventable treatable dis...
8,As a community health worker in Kenyas Homa B...
9,The progress the world has made against malari...


In [17]:
data.Tweets.iloc[0]

' 6 months ago Africa confirmed its first case of  Since then there have been 1 million cases affecting every country on'

In [13]:
data.shape

(200, 1)

In [26]:
#creating functions for Subjectivity and Polarity

def get_subjectivity(text):
    obj=TextBlob(text)
    return obj.sentiment.subjectivity

def get_polarity(text):
    obj=TextBlob(text)
    return obj.sentiment.polarity

In [27]:
data['Subjectivity']=data['Tweets'].apply(get_subjectivity)
data['Polarity']=data['Tweets'].apply(get_polarity)

In [28]:
data.head(10)

,Tweets,Subjectivity,Polarity
0,6 months ago Africa confirmed its first case ...,0.666667,0.325000
1,If were going to address climate change we nee...,0.454545,0.136364
2,Today we move one step closer toward a world w...,0.600000,0.300000
3,Thanks to the commitment of leaders from all ...,0.412500,0.100000
4,Mosquito City is home to the worlds largest ca...,0.725000,0.450000
5,Deaths from malaria have been cut by more than...,0.250000,0.125000
6,Vaccines prevent millions of deaths every yea...,0.125000,0.000000
7,Ridding the world of preventable treatable dis...,0.394886,0.284091
8,As a community health worker in Kenyas Homa B...,0.000000,0.000000
9,The progress the world has made against malari...,0.575000,0.325000


In [32]:
#creating a new column for defining whether the tweet is positive or negative or neutral based on polarity

#positive polarity stands for positive sentiment negative polarity for negative and 0 stands for neutral sentiment
def get_sentiement(polarity):
    if polarity>0:
        return 'Positive'
    elif polarity<0:
        return 'Negative'
    else :
        return 'Neutral'

In [33]:
data['Sentiment']=data['Polarity'].apply(get_sentiement)

In [35]:
data.head(10)

,Tweets,Subjectivity,Polarity,Sentiment
0,6 months ago Africa confirmed its first case ...,0.666667,0.325000,Positive
1,If were going to address climate change we nee...,0.454545,0.136364,Positive
2,Today we move one step closer toward a world w...,0.600000,0.300000,Positive
3,Thanks to the commitment of leaders from all ...,0.412500,0.100000,Positive
4,Mosquito City is home to the worlds largest ca...,0.725000,0.450000,Positive
5,Deaths from malaria have been cut by more than...,0.250000,0.125000,Positive
6,Vaccines prevent millions of deaths every yea...,0.125000,0.000000,Neutral
7,Ridding the world of preventable treatable dis...,0.394886,0.284091,Positive
8,As a community health worker in Kenyas Homa B...,0.000000,0.000000,Neutral
9,The progress the world has made against malari...,0.575000,0.325000,Positive
